### Assignment 1: Named Entity Recognition

In [1]:
experiment = 'notebook'

# os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cuda,floatX=float64"
import sys
sys.path += [new_path 
             for new_path in ['..', 
                              '../embeddings/twitter'] 
             if new_path not in sys.path]

import numpy as np
seed_number = 1337
np.random.seed(seed_number)  # for reproducibility

import pycrfsuite as crf
import matplotlib.pyplot as plt
#import common.utilities as utils

#from settings import *
from pycrfsuite import ItemSequence
from collections import Counter, defaultdict as ddict
#from embeddings.twitter.word2vecReader import Word2Vec, Vocab
from sklearn.metrics import confusion_matrix, classification_report   
from sklearn.preprocessing import LabelBinarizer

from keras.models import Model
from keras.layers import Embedding
from keras.layers import Conv1D
from keras.layers import Dropout
from keras.layers import Bidirectional
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Input
from keras.layers.merge import concatenate
from keras.layers.pooling import GlobalAveragePooling1D
from keras.optimizers import Adamax
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import re
import string
import csv
import numpy as np
import matplotlib.pyplot as plt
from itertools import groupby

### Data Preparation

In [2]:
def read_file_as_lists(filename, delimiter='\t'):
    with open(filename) as stream:
        reader = csv.reader(stream, delimiter=delimiter, quoting=csv.QUOTE_NONE)
        labeled_tokens = [zip(*g) for k, g in groupby(reader, lambda x: not [s for s in x if s.strip()]) if not k]
        tokens, labels = zip(*labeled_tokens)
        #return [list(s) for s in labeled_tokens]
        
        return [list(t) for t in tokens],[list(l) for l in labels]

In [3]:
tweets_train,   labels_train  = read_file_as_lists('/Users/rujunguan/Downloads/emerging_entities_17-master/wnut17train.conll')
tweets_dev,   labels_dev   = read_file_as_lists('/Users/rujunguan/Downloads/emerging_entities_17-master/emerging.dev.conll')
tweets_test,   labels_test   = read_file_as_lists('/Users/rujunguan/Downloads/emerging_entities_17-master/emerging.test.conll')


print(labels_train[1])

['O', 'O', 'O', 'O', 'B-group', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [4]:
# functions of sentence representations for sequence labelling
def sent2labels(sent):
    return [label for tokens, labels in sent]

def sent2tokens(sent):
    return [token for token, labels in sent]

In [5]:
X_train = tweets_train
y_train = labels_train
X_test = tweets_test
y_test = labels_test

In [6]:
import sklearn_crfsuite
crf = sklearn_crfsuite.CRF(algorithm='lbfgs', c1=0.1,c2=0.1,max_iterations=100,all_possible_transitions=True)
crf.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [7]:
labels = list(crf.classes_) 
labels.remove('O') 
print(labels)
# evaluate CRF model
from sklearn_crfsuite import metrics
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)

['B-location', 'I-location', 'B-group', 'B-corporation', 'B-person', 'B-creative-work', 'B-product', 'I-person', 'I-creative-work', 'I-corporation', 'I-group', 'I-product']


0.0313961543158042

In [8]:
def first_word(my_list):
    return [column.split(',')[0] for column in my_list]


In [9]:
# sentence representations for sequence labelling
train_sent_tokens = tweets_train
train_labels = labels_train

train_id_2_label = list(set([label for sent in train_labels for label in sent]))
train_label_2_id = {label:i for i, label in enumerate(train_id_2_label)}
print("Number of unique labels in training data:", len(train_id_2_label))

test_sent_tokens = tweets_test
test_labels = labels_test
#[item[0] for item in lst]
test_lst = []
for i in test_labels:
    a = first_word(i)
    #print(a)
    test_lst.append(a)
#print(new_lst)

test_id_2_label = list(set([label for sent in test_lst for label in sent]))
test_label_2_id = {label:i for i, label in enumerate(test_id_2_label)}
print("Number of unique labels in test data:", len(test_id_2_label))


dev_sent_tokens = tweets_dev
dev_labels = labels_dev
#[item[0] for item in lst]
dev_lst = []
for i in dev_labels:
    a = first_word(i)
    #print(a)
    dev_lst.append(a)
#print(new_lst)

dev_id_2_label = list(set([label for sent in dev_lst for label in sent]))
dev_label_2_id = {label:i for i, label in enumerate(dev_id_2_label)}
print("Number of unique labels in developing data:", len(dev_id_2_label))



def convert_labels_to_inds(sent_labels, label_2_id):
  return [label_2_id[label] for label in sent_labels]

train_label_inds = [convert_labels_to_inds(sent_labels, train_label_2_id) for sent_labels in train_labels]
#print(train_label_inds)
test_sent_tokens = tweets_test
test_labels = labels_test
test_label_inds = [convert_labels_to_inds(s, train_label_2_id) for s in test_lst]

dev_sent_tokens = tweets_dev
dev_labels = labels_dev
dev_label_inds = [convert_labels_to_inds(s, train_label_2_id) for s in dev_lst]



Number of unique labels in training data: 13
Number of unique labels in test data: 13
Number of unique labels in developing data: 13


In [10]:
#window_size = 4
window_size = 3
# converting tokenized sentence lists to vocabulary indices
id_2_word = list(set([token for sent in train_sent_tokens for token in sent])) + ["<pad>", "<unk>"]
word_2_id = {w:i for i,w in enumerate(id_2_word)}

def convert_tokens_to_inds(sentence, word_2_id):
    return [word_2_id.get(t, word_2_id["<unk>"]) for t in sentence]

# padding for windows
def pad_sentence_for_window(sentence, window_size, pad_token="<pad>"):
    return [pad_token]*window_size + sentence + [pad_token]*window_size 


In [11]:
import pprint
pp = pprint.PrettyPrinter()
print(pp)

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from functools import partial

In [12]:
# Batching sentences together with a DataLoader

def my_collate(data, window_size, word_2_id):
    """
    For some chunk of sentences and labels
        -add winow padding
        -pad for lengths using pad_sequence
        -convert our labels to one-hots
        -return padded inputs, one-hot labels, and lengths
    """
    
    x_s, y_s = zip(*data)

    # deal with input sentences as we've seen
    window_padded = [convert_tokens_to_inds(pad_sentence_for_window(sentence, window_size), word_2_id)
                                                                                  for sentence in x_s]
    # append zeros to each list of token ids in batch so that they are all the same length
    padded = nn.utils.rnn.pad_sequence([torch.LongTensor(t) for t in window_padded], batch_first=True)
    
    # convert labels to one-hots
    labels = []
    lengths = []
    for y in y_s:
        lengths.append(len(y))
        one_hot = torch.zeros(len(y), len(train_id_2_label))
        y = torch.tensor(y)
        y = y.unsqueeze(1)
        label = one_hot.scatter_(1, y, 1)
        labels.append(label)
    padded_labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)
    
    return padded.long(), padded_labels, torch.LongTensor(lengths)

In [13]:
batch_size = 2

# Shuffle True is good practice for train loaders.
# Use functools.partial to construct a partially populated collate function
train_loader = DataLoader(list(zip(train_sent_tokens, train_label_inds)), 
                            batch_size=batch_size, shuffle=True, 
                            collate_fn=partial(my_collate, window_size=2, word_2_id=word_2_id))

In [14]:
for batched_input, batched_labels, batch_lengths in train_loader:
    pp.pprint(("inputs", batched_input, batched_input.size()))
    pp.pprint(("labels", batched_labels, batched_labels.size()))
    pp.pprint(batch_lengths)
    break

('inputs',
 tensor([[14878, 14878,  6989, 14462,  6241,  7915,  1925, 10022,  8814,  9866,
         10861, 11830,  4811,  5674,  2808,  3880,  5674, 13596, 14619, 12846,
          1466,  5674, 14878, 14878,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [14878, 14878, 12247,  8886,  8814,  4004,  9687,  9477,  4623,  5674,
         10309,  6762,  1877,  8707, 10007,  5674, 10663,  4298,   992,  1835,
           711, 11851, 12820,  3164,  6415, 13564,  5674, 12254,  9534,  6964,
          6408, 14878, 14878]]),
 torch.Size([2, 33]))
('labels',
 tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 

In [15]:
class SoftmaxWordWindowClassifier(nn.Module):
    """
    A one-layer, binary word-window classifier.
    """
    def __init__(self, config, vocab_size, pad_idx=0):
        super(SoftmaxWordWindowClassifier, self).__init__()
        """
        Instance variables.
        """
        self.window_size = 2*config["half_window"]+1
        self.embed_dim = config["embed_dim"]
        self.hidden_dim = config["hidden_dim"]
        self.num_classes = config["num_classes"]
        self.freeze_embeddings = config["freeze_embeddings"]
        
        """
        Embedding layer
        -model holds an embedding for each layer in our vocab
        -sets aside a special index in the embedding matrix for padding vector (of zeros)
        -by default, embeddings are parameters (so gradients pass through them)
        """
        self.embed_layer = nn.Embedding(vocab_size, self.embed_dim, padding_idx=pad_idx)
        if self.freeze_embeddings:
            self.embed_layer.weight.requires_grad = False
        
        """
        Hidden layer
        -we want to map embedded word windows of dim (window_size+1)*self.embed_dim to a hidden layer.
        -nn.Sequential allows you to efficiently specify sequentially structured models
            -first the linear transformation is evoked on the embedded word windows
            -next the nonlinear transformation tanh is evoked.
        """
       # self.hidden_layer = nn.Sequential(nn.Linear(self.window_size*self.embed_dim, 
       #                                             self.hidden_dim), 
       #                                   nn.Tanh(),
       #                                   nn.Linear(self.hidden_dim, 
       #                                            2), 
       #                                   nn.Tanh(),
       #                                   nn.Linear(2,
       #                                           2),
       #                                  nn.Tanh()
       #                                 )
       
        
        """
        Output layer
        -we want to map elements of the output layer (of size self.hidden dim) to a number of classes.
        """
       # self.output_layer = nn.Linear(2, self.num_classes)
        
        """
        Softmax
        -The final step of the softmax classifier: mapping final hidden layer to class scores.
        -pytorch has both logsoftmax and softmax functions (and many others)
        -since our loss is the negative LOG likelihood, we use logsoftmax
        -technically you can take the softmax, and take the log but PyTorch's implementation
         is optimized to avoid numerical underflow issues.
        """
        
        self.hidden_layer = nn.Sequential(nn.Linear(self.window_size*self.embed_dim, 
                                                    self.hidden_dim), 
                                          nn.Tanh())
        
        """
        Output layer
        -we want to map elements of the output layer (of size self.hidden dim) to a number of classes.
        """
        self.output_layer = nn.Linear(self.hidden_dim, self.num_classes)
        
        
       #self.log_softmax = nn.LogSoftmax(dim=2)
        self.log_softmax = nn.LogSoftmax(dim=2)
    def forward(self, inputs):
        """
        Let B:= batch_size
            L:= window-padded sentence length
            D:= self.embed_dim
            S:= self.window_size
            H:= self.hidden_dim
            
        inputs: a (B, L) tensor of token indices
        """
        B, L = inputs.size()
        
        """
        Reshaping.
        Takes in a (B, L) LongTensor
        Outputs a (B, L~, S) LongTensor
        """
        # Fist, get our word windows for each word in our input.
        token_windows = inputs.unfold(1, self.window_size, 1)
        _, adjusted_length, _ = token_windows.size()
        
        # Good idea to do internal tensor-size sanity checks, at the least in comments!
        assert token_windows.size() == (B, adjusted_length, self.window_size)
        
        """
        Embedding.
        Takes in a torch.LongTensor of size (B, L~, S) 
        Outputs a (B, L~, S, D) FloatTensor.
        """
        embedded_windows = self.embed_layer(token_windows)
        
        """
        Reshaping.
        Takes in a (B, L~, S, D) FloatTensor.
        Resizes it into a (B, L~, S*D) FloatTensor.
        -1 argument "infers" what the last dimension should be based on leftover axes.
        """
        embedded_windows = embedded_windows.view(B, adjusted_length, -1)
        
        """
        Layer 1.
        Takes in a (B, L~, S*D) FloatTensor.
        Resizes it into a (B, L~, H) FloatTensor
        """
        layer_1 = self.hidden_layer(embedded_windows)
        
        """
        Layer 2
        Takes in a (B, L~, H) FloatTensor.
        Resizes it into a (B, L~, 2) FloatTensor.
        """
        
       # linear2 = self.hidden_layer(layer_1)
        
        output = self.output_layer(layer_1)
        
        """
        Softmax.
        Takes in a (B, L~, 2) FloatTensor of unnormalized class scores.
        Outputs a (B, L~, 2) FloatTensor of (log-)normalized class scores.
        """
        output = self.log_softmax(output)
        
        return output

In [16]:
def loss_function(outputs, labels, lengths):
    """Computes negative LL loss on a batch of model predictions."""
    B, L, num_classes = outputs.size()
    num_elems = lengths.sum().float()
        
    # get only the values with non-zero labels
    loss = outputs*labels
    
    # rescale average
    return -loss.sum() / num_elems

In [17]:
def train_epoch(loss_function, optimizer, model, train_data):
    
    ## For each batch, we must reset the gradients
    ## stored by the model.   
    total_loss = 0
    for batch, labels, lengths in train_data:
        # clear gradients
        optimizer.zero_grad()
        # evoke model in training mode on batch
        outputs = model.forward(batch)
        # compute loss w.r.t batch
        loss = loss_function(outputs, labels, lengths)
        # pass gradients back, startiing on loss value
        loss.backward()
        # update parameters
        optimizer.step()
        total_loss += loss.item()
    
    # return the total to keep track of how you did this time around
    return total_loss

In [24]:
config = {"batch_size": 4,
          "half_window": 2,
          "embed_dim": 15,
          "hidden_dim": 25,
          "num_classes": 13,
          "freeze_embeddings": False,
         }
learning_rate = .000002
num_epochs = 10
model = SoftmaxWordWindowClassifier(config, len(word_2_id))
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [25]:
for epoch in range(num_epochs):
    epoch_loss = train_epoch(loss_function, optimizer, model, train_loader)
    print(epoch, epoch_loss)

0 4277.870305299759
1 4270.20210814476
2 4262.548305273056
3 4256.0604157447815
4 4249.405846595764
5 4241.872244119644
6 4235.232044219971
7 4228.2011914253235
8 4220.95866894722
9 4214.118665456772


### Evaluation
There is much more O entities in data set, but we’re more interested in other entities. To account for this we’ll use averaged F1 score computed for all labels except for O. sklearn-crfsuite.metrics package provides some useful metrics for sequence classification task, including this one.

In [26]:
test_loader = DataLoader(list(zip(test_sent_tokens, test_label_inds)), 
                            batch_size=batch_size, shuffle=False, 
                            collate_fn=partial(my_collate, window_size=2, word_2_id=word_2_id))

In [27]:
test_outputs = []
for test_instance, labs, _ in test_loader:
    outputs_full = model.forward(test_instance)
    outputs = torch.argmax(outputs_full, dim=2)
    for i in range(outputs.size(0)):
      test_outputs.append(outputs[i].tolist())


In [28]:
y_test = test_labels
y_pred = []
for test, pred in zip(test_labels, test_outputs):
  y_pred.append([train_id_2_label[id] for id in pred[:len(test)]])

assert len(y_pred) == len(y_test), '{} vs. {}'.format(len(y_pred), len(y_test))
for i, pred, test in zip(list(range(len(y_pred))), y_pred, y_test):
  assert len(pred) == len(test), '{}: {} vs. {}'.format(i, len(pred), len(test))


In [29]:
print(test_id_2_label)

['B-creative-work', 'B-person', 'I-creative-work', 'B-group', 'B-location', 'I-group', 'B-corporation', 'I-location', 'I-corporation', 'B-product', 'O', 'I-product', 'I-person']


In [30]:
train_id_2_label.remove('O')

In [31]:
# evaluate CRF model
#!pip3 install sklearn-crfsuite
from sklearn_crfsuite import metrics

metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=train_id_2_label)

0.007698163873677287

In [32]:
y_pred[1]

['I-corporation',
 'I-location',
 'O',
 'O',
 'O',
 'O',
 'I-corporation',
 'B-group',
 'I-corporation',
 'I-corporation',
 'O',
 'I-corporation',
 'I-corporation',
 'I-corporation',
 'B-group',
 'I-corporation',
 'I-person',
 'O',
 'B-product',
 'I-person',
 'I-corporation',
 'I-corporation',
 'O',
 'B-group',
 'I-corporation',
 'O',
 'I-corporation',
 'I-person',
 'B-location',
 'B-location',
 'I-corporation',
 'B-group',
 'B-person',
 'B-product']

In [382]:
y_test

[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-location',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-location',
  'I-location',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-group',
  'I-group',
  'I-group',
  'O',

### Inspect per-class results in more detail:

In [33]:
# group B and I results
sorted_labels = sorted(
    train_id_2_label,
    key=lambda name: (name[1:], name[0])
)

print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=['B-corporation', 'I-corporation', 'B-creative-work', 'I-creative-work', 'B-group', 'I-group', 'B-location', 'I-location', 'B-person', 'I-person', 'B-product', 'I-product'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


                 precision    recall  f1-score   support

  B-corporation      0.002     0.032     0.003        62
  I-corporation      0.001     0.250     0.001        16
B-creative-work      0.006     0.053     0.012       131
I-creative-work      0.008     0.012     0.010       172
        B-group      0.007     0.165     0.014        97
        I-group      0.001     0.026     0.002        39
     B-location      0.003     0.019     0.005       104
     I-location      0.001     0.033     0.002        30
       B-person      0.010     0.006     0.008       333
       I-person      0.005     0.086     0.009        93
      B-product      0.002     0.060     0.003        50
      I-product      0.000     0.000     0.000        36

      micro avg      0.003     0.041     0.005      1163
      macro avg      0.004     0.062     0.006      1163
   weighted avg      0.006     0.041     0.008      1163



## Evaluation with dev_loader

In [59]:
train_sent_tokens = tweets_train
train_labels = labels_train

train_id_2_label = list(set([label for sent in train_labels for label in sent]))
train_label_2_id = {label:i for i, label in enumerate(train_id_2_label)}
print("Number of unique labels in training data:", len(train_id_2_label))

Number of unique labels in training data: 13


In [50]:
train_loader = DataLoader(list(zip(train_sent_tokens, train_label_inds)), 
                            batch_size=batch_size, shuffle=True, 
                            collate_fn=partial(my_collate, window_size=2, word_2_id=word_2_id))

In [51]:
config = {"batch_size": 4,
          "half_window": 2,
          "embed_dim": 25,
          "hidden_dim": 25,
          "num_classes": 13,
          "freeze_embeddings": False,
         }
learning_rate = .0002
num_epochs = 30
model = SoftmaxWordWindowClassifier(config, len(word_2_id))
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [52]:
for epoch in range(num_epochs):
    epoch_loss = train_epoch(loss_function, optimizer, model, train_loader)
    print(epoch, epoch_loss)

0 4235.058019161224
1 3518.7618800401688
2 2810.0788885354996
3 2191.2559828162193
4 1723.103898882866
5 1399.1607606261969
6 1178.992493905127
7 1027.4241096302867
8 917.0839348882437
9 841.2435817755759
10 778.8017147667706
11 736.2713016234338
12 700.1137227974832
13 679.3445205483586
14 653.467975653708
15 635.5484840795398
16 622.3189433477819
17 615.9979454744607
18 602.9911899901927
19 604.7205450199544
20 585.9775352142751
21 578.0985265113413
22 568.7813751734793
23 566.3712784033269
24 569.4907760061324
25 566.3495938163251
26 557.1913135573268
27 563.1647447608411
28 553.3110638074577
29 552.6112159658223


In [55]:
#Using validation set for hyper paremeter tuning
dev_loader = DataLoader(list(zip(dev_sent_tokens, dev_label_inds)), 
                            batch_size=batch_size, shuffle=False, 
                            collate_fn=partial(my_collate, window_size=2, word_2_id=word_2_id))


In [56]:
dev_outputs = []
for dev_instance, labs, _ in dev_loader:
    outputs_full = model.forward(dev_instance)
    outputs = torch.argmax(outputs_full, dim=2)
    for i in range(outputs.size(0)):
      dev_outputs.append(outputs[i].tolist())

In [57]:
y_dev = dev_labels
y_pred = []
for dev, pred in zip(dev_labels, dev_outputs):
  y_pred.append([train_id_2_label[id] for id in pred[:len(dev)]])

assert len(y_pred) == len(y_dev), '{} vs. {}'.format(len(y_pred), len(y_dev))
for i, pred, dev in zip(list(range(len(y_pred))), y_pred, y_dev):
  assert len(pred) == len(dev), '{}: {} vs. {}'.format(i, len(pred), len(dev))

In [58]:
# group B and I results
sorted_labels = sorted(
    train_id_2_label,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_dev, y_pred, labels=sorted_labels, digits=3
))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=['O', 'B-corporation', 'I-corporation', 'B-creative-work', 'I-creative-work', 'B-group', 'I-group', 'B-location', 'I-location', 'B-person', 'I-person', 'B-product', 'I-product'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                 precision    recall  f1-score   support

              O      0.921     1.000     0.959     14483
  B-corporation      0.000     0.000     0.000        34
  I-corporation      0.000     0.000     0.000        12
B-creative-work      0.000     0.000     0.000       105
I-creative-work      0.000     0.000     0.000       133
        B-group      0.000     0.000     0.000        39
        I-group      0.000     0.000     0.000        25
     B-location      0.000     0.000     0.000        74
     I-location      0.000     0.000     0.000        33
       B-person      0.000     0.000     0.000       470
       I-person      0.000     0.000     0.000       117
      B-product      0.000     0.000     0.000       114
      I-product      0.000     0.000     0.000        94

       accuracy                          0.921     15733
      macro avg      0.071     0.077     0.074     15733
   weighted avg      0.847     0.921     0.882     15733

